In [18]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict, Literal
from IPython.display import Image, display
from langgraph.prebuilt import create_react_agent
from langchain import hub
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import MultionToolkit
import json
from langchain.output_parsers import PydanticOutputParser
# from langchain_aws import ChatBedrockConverse
import os
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface import ChatHuggingFace

load_dotenv()

True

In [ ]:
AWS_ACCESS_KEY_ID = 
AWS_SECRET_ACCESS_KEY = 

In [4]:
Industry = Literal[
    "advertising-marketing",
    "automotive",
    "retail-consumer-goods",
    "energy-utilities",
    "financial-services",
    "gametech",
    "health",
    "industrial",
    "manufacturing",
    "media",
    "telecom",
]

class State(TypedDict):
    url: str
    bio: str | None
    industry: Industry | None

In [11]:
from pydantic import BaseModel
class BioInfo(BaseModel):
    industry: Industry | None 
    about: str | None

In [15]:
repo_id = "deepseek-ai/DeepSeek-R1-0528"

llm_endpoint = HuggingFaceEndpoint(
    repo_id=repo_id,
    # max_length=128,
    temperature= 1,
    provider="auto",  # set your provider here hf.co/settings/inference-providers
)
llm = ChatHuggingFace(llm=llm_endpoint)

In [20]:
# this agent to generate company bio and recognise industry
prompt = """
    Extract biographical and industry information from the given URL.
    
    For biographical information, return JSON in this format:
    {bio_parser.get_format_instructions()}
    
    For industry information, return JSON in this format:
    {industry_parser.get_format_instructions()}
    
    Return the results as two separate JSON objects labeled as "bio" and "industry
"""
# llm = ChatBedrockConverse(
#     model_id="anthropic.claude-3-5-sonnet-latest-v1:0"
# )
web_toolkit = MultionToolkit()
tools = web_toolkit.get_tools()
bio_agent = create_react_agent(
    model = llm,
    tools = tools,
    prompt = prompt
    
)

def bio_agent_node(state: State):
    bio_parser = PydanticOutputParser(pydantic_object=BioInfo)
    result = bio_agent.invoke({"messages":[
        {"role": "user", "content": state["url"]}
    ]})
    print(result)

    response_text = result["messages"][-1]["content"]
    try:
        bio_data = bio_parser.parse(extract_bio_json(response_text))
        return {"bio": bio_data.about, "industry": bio_data.industry}
    except Exception as e:
        return {"bio": None, "industry": None}


In [21]:
bio_agent_node({"url": "https://tz.net/"})

{'messages': [HumanMessage(content='https://tz.net/', additional_kwargs={}, response_metadata={}, id='ae45e5db-3c70-4b19-83e4-2ad66acad649'), AIMessage(content='<think>\nWe are given a URL: https://tz.net/\n We are to summarize the company based on the content of the website.\n Steps:\n 1. We will use the WebScraper tool to fetch the content of the homepage and any key pages (like About, Services, etc.) if necessary.\n 2. Since the tool can only fetch one page at a time, we start with the homepage.\n 3. We will look for key information such as:\n    - Company name\n    - Tagline or mission statement\n    - Services or products offered\n    - Key differentiators or unique selling points\n    - Any other notable information (like history, team, etc.)\n 4. If the homepage doesn\'t have enough information, we might need to navigate to an "About" page or similar.\n\n However, note that the tool only returns the text content. We can also look for links to other pages (like "About") and then 

{'bio': None, 'industry': None}

In [ ]:
def industry_news_node(state: State):
    prompt = """
    """
    search = DuckDuckGoSearchRun()
    results = search.invoke(prompt + state["industry"])
    return state

In [ ]:
from urllib.parse import quote

def usecase_node(state: State):
    path = quote(str(state.get('industry', '')).strip('/'))
    loader = WebBaseLoader(f"https://aws.amazon.com/{path}/")
    return state

In [ ]:
def industry_problems (state: State):
    prompt = """
    """
    search = DuckDuckGoSearchRun()
    results = search.invoke(prompt + state["industry"])
    return state